그리드서치

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.class_weight import compute_class_weight

# 데이터 로드 (파일 경로 수정 필요)
data = pd.read_csv('TA_cleaned.csv')  # 실제 파일 경로로 수정

# 1. 클래스 9 제거
data_no_class_9 = data[data['accTypeD'] != 9]

# 2. 클래스 2, 6, 7, 8 통합
data_no_class_9['accTypeD_merged_combined'] = data_no_class_9['accTypeD'].replace({6: 2, 7: 2, 8: 2})

# 3. 타겟 변수와 피처 정의
features_with_grid = ['hour', 'is_holiday', 'road_form_class', 'road_formD', 'carFLg',
                      'carClassF', 'carClassVic', 'lo_crd', 'la_crd']
data_no_class_9['grid_lat'] = (data_no_class_9['la_crd'] // 0.02) * 0.02  # 2km 격자화
data_no_class_9['grid_lon'] = (data_no_class_9['lo_crd'] // 0.02) * 0.02
data_no_class_9['grid_id'] = data_no_class_9['grid_lat'].astype(str) + '_' + data_no_class_9['grid_lon'].astype(str)

y_combined = data_no_class_9['accTypeD_merged_combined']
X_combined = pd.get_dummies(
    data_no_class_9[features_with_grid + ['grid_id']], drop_first=True
)

# 4. 데이터 분리
X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(
    X_combined, y_combined, test_size=0.2, random_state=42
)

# 5. 클래스 가중치 계산 및 통합된 클래스(2)의 가중치 증가
class_weights_combined = compute_class_weight(
    class_weight="balanced", classes=np.unique(y_train_combined), y=y_train_combined
)
class_weight_dict_combined = {
    i: (weight * 2 if i == 2 else weight) for i, weight in zip(np.unique(y_train_combined), class_weights_combined)
}

# 6. 하이퍼파라미터 범위 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 7. GridSearchCV로 최적의 하이퍼파라미터 탐색
rf_grid_model = RandomForestClassifier(random_state=42, class_weight=class_weight_dict_combined)

grid_search = GridSearchCV(estimator=rf_grid_model, param_grid=param_grid,
                           cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train_combined, y_train_combined)

# 8. 최적 하이퍼파라미터 출력
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# 9. 최적 하이퍼파라미터로 학습된 모델 평가
best_rf_model = grid_search.best_estimator_
y_pred_best_rf = best_rf_model.predict(X_test_combined)

# 10. 성능 평가
accuracy_best_rf = accuracy_score(y_test_combined, y_pred_best_rf)
report_best_rf = classification_report(y_test_combined, y_pred_best_rf, target_names=[str(cls) for cls in np.unique(y_train_combined)])

print("Accuracy:", accuracy_best_rf)
print("Classification Report:\n", report_best_rf)


Fitting 3 folds for each of 81 candidates, totalling 243 fits


<ipython-input-4-fb14e58d485b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_class_9['accTypeD_merged_combined'] = data_no_class_9['accTypeD'].replace({6: 2, 7: 2, 8: 2})
<ipython-input-4-fb14e58d485b>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_class_9['grid_lat'] = (data_no_class_9['la_crd'] // 0.02) * 0.02  # 2km 격자화
<ipython-input-4-fb14e58d485b>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

Best Parameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
Accuracy: 0.5822222222222222
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.57      0.63        83
           1       0.50      0.11      0.17        19
           2       0.45      0.80      0.58       126
           3       0.57      0.78      0.66        50
           4       0.87      0.44      0.58       166
           5       0.00      0.00      0.00         6

    accuracy                           0.58       450
   macro avg       0.52      0.45      0.44       450
weighted avg       0.66      0.58      0.57       450



In [ ]:
#한글
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

# 한글 폰트 설치
!apt-get -qq install fonts-nanum

# 설치된 폰트 설정
# fm._rebuild()  # Remove this line as _rebuild is a private method and might not be available.
fm.fontManager.addfont('/usr/share/fonts/truetype/nanum/NanumGothic.ttf') # Add this line to register NanumGothic font manually
plt.rc('font', family='NanumGothic')  # 나눔고딕 폰트 설정

Selecting previously unselected package fonts-nanum.
(Reading database ... 123630 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

# 데이터 로드 (파일 경로 수정 필요)
data = pd.read_csv('TA_cleaned.csv')  # 파일 경로를 실제 경로로 수정

In [ ]:
# 1. 클래스 9 제거
data_no_class_9 = data[data['accTypeD'] != 9]

# 2. 클래스 2와 5 통합
data_no_class_9['accTypeD_merged'] = data_no_class_9['accTypeD'].replace({5: 2})

# 3. 타겟 변수와 피처 정의
features_with_grid = ['hour', 'is_holiday', 'road_form_class', 'road_formD', 'carFLg',
                      'carClassF', 'carClassVic', 'lo_crd', 'la_crd']  # 필요한 피처 추가
data_no_class_9['grid_lat'] = (data_no_class_9['la_crd'] // 0.02) * 0.02  # 2km 격자화
data_no_class_9['grid_lon'] = (data_no_class_9['lo_crd'] // 0.02) * 0.02
data_no_class_9['grid_id'] = data_no_class_9['grid_lat'].astype(str) + '_' + data_no_class_9['grid_lon'].astype(str)

y = data_no_class_9['accTypeD_merged']
X = pd.get_dummies(data_no_class_9[features_with_grid + ['grid_id']], drop_first=True)

# 4. 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. 클래스 가중치 계산 및 조정
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
class_weight_dict = {cls: weight * 2 if cls == 2 else weight for cls, weight in zip(np.unique(y_train), class_weights)}

# 6. 모델 학습
model = RandomForestClassifier(random_state=42, class_weight=class_weight_dict)
model.fit(X_train, y_train)

# 7. 테스트 데이터 예측
y_pred = model.predict(X_test)

# 8. 성능 평가
report = classification_report(y_test, y_pred, target_names=[str(cls) for cls in np.unique(y_train)])
conf_matrix = confusion_matrix(y_test, y_pred, labels=np.unique(y_train))

# 결과 출력
print("Classification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)

# 클래스 2(TP, FP, FN) 세부 평가
class_2_index = list(np.unique(y_test)).index(2)
class_2_metrics = {
    "True Positives (TP)": conf_matrix[class_2_index, class_2_index],
    "False Positives (FP)": conf_matrix[:, class_2_index].sum() - conf_matrix[class_2_index, class_2_index],
    "False Negatives (FN)": conf_matrix[class_2_index, :].sum() - conf_matrix[class_2_index, class_2_index],
    "Total Test Cases": y_test.value_counts().get(2, 0)
}
print("\nClass 2 Metrics:\n", class_2_metrics)

<ipython-input-2-691140e5b7ea>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_class_9['accTypeD_merged'] = data_no_class_9['accTypeD'].replace({5: 2})
<ipython-input-2-691140e5b7ea>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_class_9['grid_lat'] = (data_no_class_9['la_crd'] // 0.02) * 0.02  # 2km 격자화
<ipython-input-2-691140e5b7ea>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.88      0.73        83
           1       1.00      0.05      0.10        19
           2       0.00      0.00      0.00        14
           3       0.74      0.56      0.64        50
           4       0.61      0.88      0.72       166
           6       0.22      0.07      0.11        28
           7       0.35      0.26      0.30        31
           8       0.25      0.10      0.14        59

    accuracy                           0.59       450
   macro avg       0.47      0.35      0.34       450
weighted avg       0.54      0.59      0.52       450


Confusion Matrix:
 [[ 73   0   0   0   2   0   0   8]
 [ 15   1   0   0   0   0   0   3]
 [  9   0   0   0   1   0   0   4]
 [  0   0   0  28  22   0   0   0]
 [  1   0   0   5 146   3  10   1]
 [  0   0   0   0  21   2   4   1]
 [  0   0   0   0  20   2   8   1]
 [ 18   0   0   5  27   2   1   6]]

Class 2 Metrics:
 {'Tru

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


XGBoost와 LightGBM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# XGBoost 모델 학습 및 평가
xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train_combined, y_train_combined)
y_pred_xgb = xgb_model.predict(X_test_combined)
accuracy_xgb = accuracy_score(y_test_combined, y_pred_xgb)
report_xgb = classification_report(y_test_combined, y_pred_xgb, target_names=[str(cls) for cls in np.unique(y_train_combined)])

print("XGBoost Accuracy:", accuracy_xgb)
print("XGBoost Classification Report:\n", report_xgb)

# LightGBM 모델 학습 및 평가
lgbm_model = LGBMClassifier(random_state=42)
lgbm_model.fit(X_train_combined, y_train_combined)
y_pred_lgbm = lgbm_model.predict(X_test_combined)
accuracy_lgbm = accuracy_score(y_test_combined, y_pred_lgbm)
report_lgbm = classification_report(y_test_combined, y_pred_lgbm, target_names=[str(cls) for cls in np.unique(y_train_combined)])

print("LightGBM Accuracy:", accuracy_lgbm)
print("LightGBM Classification Report:\n", report_lgbm)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:52:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.6044444444444445
XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.67      0.64        83
           1       0.20      0.11      0.14        19
           2       0.50      0.53      0.52       126
           3       0.60      0.60      0.60        50
           4       0.71      0.70      0.71       166
           5       0.00      0.00      0.00         6

    accuracy                           0.60       450
   macro avg       0.44      0.44      0.43       450
weighted avg       0.59      0.60      0.60       450

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000608 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 582
[LightGBM] [Info] Number of data points in the train set: 1796, number of used features: 14
[LightGBM] [Info] Start training from

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


--------------------------------------
LightGBM 모델 최적 선택

In [ ]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from lightgbm import LGBMClassifier

# 1. 데이터 로드
data = pd.read_csv('TA_cleaned.csv')  # 실제 파일 경로로 수정

# 2. 클래스 9 제거
data_no_class_9 = data[data['accTypeD'] != 9]

# 3. 클래스 2, 6, 7, 8 통합
data_no_class_9['accTypeD_merged_combined'] = data_no_class_9['accTypeD'].replace({6: 2, 7: 2, 8: 2})

# 4. 격자화 및 피처 정의
features_with_grid = ['hour', 'is_holiday', 'road_form_class', 'road_formD', 'carFLg',
                      'carClassF', 'carClassVic', 'lo_crd', 'la_crd']
data_no_class_9['grid_lat'] = (data_no_class_9['la_crd'] // 0.02) * 0.02  # 2km 격자화
data_no_class_9['grid_lon'] = (data_no_class_9['lo_crd'] // 0.02) * 0.02
data_no_class_9['grid_id'] = data_no_class_9['grid_lat'].astype(str) + '_' + data_no_class_9['grid_lon'].astype(str)

y_combined = data_no_class_9['accTypeD_merged_combined']
X_combined = pd.get_dummies(
    data_no_class_9[features_with_grid + ['grid_id']], drop_first=True
)

# 5. 데이터 분리
X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(
    X_combined, y_combined, test_size=0.2, random_state=42
)

# 6. 클래스 가중치 계산 및 통합된 클래스(2)의 가중치 증가
class_weights_combined = compute_class_weight(
    class_weight="balanced", classes=np.unique(y_train_combined), y=y_train_combined
)
class_weight_dict_combined = {
    i: (weight * 2 if i == 2 else weight) for i, weight in zip(np.unique(y_train_combined), class_weights_combined)
}

# 7. LightGBM 모델 학습 및 평가
lgbm_model = LGBMClassifier(random_state=42, class_weight=class_weight_dict_combined)
lgbm_model.fit(X_train_combined, y_train_combined)
y_pred_lgbm = lgbm_model.predict(X_test_combined)

# 8. 성능 평가
accuracy_lgbm = accuracy_score(y_test_combined, y_pred_lgbm)
report_lgbm = classification_report(y_test_combined, y_pred_lgbm, target_names=[str(cls) for cls in np.unique(y_train_combined)])

# 결과 출력
print("LightGBM Accuracy:", accuracy_lgbm)
print("LightGBM Classification Report:\n", report_lgbm)


<ipython-input-6-4c852ffa6d3d>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_class_9['accTypeD_merged_combined'] = data_no_class_9['accTypeD'].replace({6: 2, 7: 2, 8: 2})
<ipython-input-6-4c852ffa6d3d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_class_9['grid_lat'] = (data_no_class_9['la_crd'] // 0.02) * 0.02  # 2km 격자화
<ipython-input-6-4c852ffa6d3d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000289 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 582
[LightGBM] [Info] Number of data points in the train set: 1796, number of used features: 14
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.252763
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
LightGBM Accuracy: 0.5955

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


------------------------------
1. 입력 피처 (Features)
모델의 입력 피처로 사용된 변수는 아래와 같습니다:

- hour: 사고 발생 시간.

- is_holiday: 공휴일 여부.

- road_form_class: 도로 형태를 나타내는 변수.

- road_formD: 상세 도로 형태를 나타내는 변수.

- carFLg: 사고 차량의 플래그.

- carClassF: 사고 차량의 분류.

- carClassVic: 피해 차량의 분류.

- lo_crd: 사고 위치의 경도 (longitudinal coordinate).

- la_crd: 사고 위치의 위도 (latitudinal coordinate).

- grid_id: 사고 위치를 2km x 2km 간격으로 격자화하여 생성한 범주형 변수.

 - 이 피처들은 모델의 입력 데이터(X_combined)로 사용됩니다.

-------------------------

 예측 대상 (Target)

이는 원래의 사고 유형(accTypeD)에서 일부 클래스를 통합한 변수입니다.
최종적으로 아래와 같은 클래스가 예측 대상입니다

  1. 0 - 횡단중
  2. 1 - 차도통행중
  3. 2 (통합된 클래스)
    - 원래 클래스(2, 6, 7, 8)
      - 2 : 길가장자리구역통행중
      - 6 : 도로이탈
      - 7 : 전도전복
      - 8 : 기타
  4. 3 : 추돌
  5. 4 : 충돌

  --------------------
  3. 삭제된 클래스
  - 5번, 9번 클래스 샘플 수 너무 적어서 그냥 삭제~~
    - 5 : 보도통행중
    - 9 : 쓰잘때기 묶어뒀던거

In [ ]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from lightgbm import LGBMClassifier

# 1. 데이터 로드
data = pd.read_csv('TA_cleaned.csv')  # 실제 파일 경로로 수정

# 2. 클래스 9 및 클래스 5 삭제
data_filtered = data[data['accTypeD'].isin([0, 1, 2, 3, 4, 6, 7, 8])]

# 3. 클래스 2, 6, 7, 8 통합
data_filtered['accTypeD_merged_combined'] = data_filtered['accTypeD'].replace({6: 2, 7: 2, 8: 2})

# 4. 격자화 및 피처 정의
features_with_grid = ['hour', 'is_holiday', 'road_form_class', 'road_formD', 'carFLg',
                      'carClassF', 'carClassVic', 'lo_crd', 'la_crd']
data_filtered['grid_lat'] = (data_filtered['la_crd'] // 0.02) * 0.02  # 2km 격자화
data_filtered['grid_lon'] = (data_filtered['lo_crd'] // 0.02) * 0.02
data_filtered['grid_id'] = data_filtered['grid_lat'].astype(str) + '_' + data_filtered['grid_lon'].astype(str)

y_combined = data_filtered['accTypeD_merged_combined']
X_combined = pd.get_dummies(
    data_filtered[features_with_grid + ['grid_id']], drop_first=True
)

# 5. 데이터 분리
X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(
    X_combined, y_combined, test_size=0.2, random_state=42
)

# 6. 클래스 가중치 계산 및 통합된 클래스(2)의 가중치 증가
class_weights_combined = compute_class_weight(
    class_weight="balanced", classes=np.unique(y_train_combined), y=y_train_combined
)
class_weight_dict_combined = {
    i: (weight * 2 if i == 2 else weight) for i, weight in zip(np.unique(y_train_combined), class_weights_combined)
}

# 7. LightGBM 모델 학습 및 평가
lgbm_model = LGBMClassifier(random_state=42, class_weight=class_weight_dict_combined)
lgbm_model.fit(X_train_combined, y_train_combined)
y_pred_lgbm = lgbm_model.predict(X_test_combined)

# 8. 성능 평가
accuracy_lgbm = accuracy_score(y_test_combined, y_pred_lgbm)
report_lgbm = classification_report(
    y_test_combined, y_pred_lgbm, target_names=[str(cls) for cls in np.unique(y_train_combined)]
)

# 결과 출력
print("LightGBM Accuracy:", accuracy_lgbm)
print("LightGBM Classification Report:\n", report_lgbm)


<ipython-input-7-45a9a54f19b6>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['accTypeD_merged_combined'] = data_filtered['accTypeD'].replace({6: 2, 7: 2, 8: 2})
<ipython-input-7-45a9a54f19b6>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['grid_lat'] = (data_filtered['la_crd'] // 0.02) * 0.02  # 2km 격자화
<ipython-input-7-45a9a54f19b6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000208 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 581
[LightGBM] [Info] Number of data points in the train set: 1779, number of used features: 14
[LightGBM] [Info] Start training from score -1.791760
[LightGBM] [Info] Start training from score -1.791760
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
LightGBM Accuracy: 0.6112359550561798
LightGBM Classification Report:
               precision    recall  f1-score   support

           0       0.59     